## Author: Aditya Sundar
## Email: aditya.1094@gmail.com

## <font color='blue'>Electric vehicles (EVs) are predicted to be an integral part of modern, sustainable economies.  Replacing combustion engines with batteries reduces our carbon footprint and improves air quality. However, the current costs associated with researching & manufacturing batteries makes EVs more expensive. As such, it is crucial for the buyer to be informed about the pros and cons of purchasing EVs. The project is intended to help vehicle buyers make informed selections of EVs, by providing </font> <font color='red'> 1) interactive dashboards to explore the performance metrics of commercial EVs </font><font color='blue'>and<font> <font color='red'>2) models to estimate vehicle costs.</font>

### Note: There is a heading before each cell or group of cells. Headings in black font denote code for data retrieval and cleaning. Headings in <font color='red'>red</font> font denote visualization codes that can be run directly using processed data.

In [4]:
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")
import time
import numpy as np
import requests
from bs4 import BeautifulSoup
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import ipywidgets as widgets
import unicodedata
import re
import seaborn as sns
import inspect
import matplotlib.pyplot as plt

# Web scraping to get data for electric vehicles (EV) sales in the USA
Data includes various types of electric vehicles, manufactured by several automobile companies, showing purchasing trends from 2000 to 2022.

In [12]:
url = 'https://www.atlasevhub.com/materials/state-ev-registration-data/#data'
reqs = requests.get(url)
soup = BeautifulSoup(reqs.text, 'html.parser')

links=[link.get('href') for link in soup.find_all('a') ]
data=[i for i in links if len(i.split('/'))>1 and i.split('/')[1]=='public']
data=[i for i in data if i.split('.')[-1]=='csv']
state_name=[i.split('/')[3].split('_')[0].upper() for i in data]

dt=[]
for i in [data[-2]]:
    dt.append(pd.read_csv('https://www.atlasevhub.com/'+i))